# Create Targets From Land Survey Tide Staff Excel File

This notebook will take an excell file from the land survey department and export a target file for Hypack. 

First download a copy of the SWG Tide Staffs Excel file. Delete any rows that have strikethrough text formatting. Then delete any extran columns. The only columns that are needed are the name, x, y, date. 
The date is not really needed, but it kept so that it gets written to the .tgt file. However, for some reason the date does not populate in Hypack when importing. 



In [31]:
import pandas as pd 
from datetime import date
import csv

In [37]:
# Initialization

# Set the working directory
WD = "/home/roger/Documents/Python/"
# enter the location of the xcell spreadsheet that has all the tide staffs
Targets = "/home/roger/Documents/Python/SWG_Tide_Staffs_Testing.xlsx"

# Get the date today 
Todays_Date = date.today()
Todays_Date = Todays_Date.strftime("%Y%m%d")

# Create the output filename
Output_File = f"SWG_Tide_Staffs_{Todays_Date}.tgt"
#print (Output_File)

In [38]:
# The spreadsheet contains 3 sheets, so they will need to be read in separately and then combined.
df1 = pd.read_excel(Targets, "Bay City Tide Staffs", index_col=None, na_values=["NA"])
df2 = pd.read_excel(Targets, "Galveston Tide Staffs", index_col=None, na_values=["NA"])
df3 = pd.read_excel(Targets, "La Porte Tide Staffs", index_col=None, na_values=["NA"])

# Combining the 3 sheets
frames = [df1, df2, df3]
df = pd.concat(frames)
df

,Staff Name,Location,Unnamed: 2,NAVD88 to MLLW Conversion,Top of 4' Staff Elevation,Unnamed: 5,Date Set or Checked,Channel,Notes
0,NaN,Easting,Northing,NaN,NAVD88,MLLW,NaT,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,Brazos Harbor,3142320.1,13541514.3,-0.6,3.4,NaN,2023-09-18,Freeport,Set from RTK(120epochs)r12i_ 9/18/2023
3,Freeport Gate (guillotine),3139199.6,13548074.5,-0.6,3.4,4,2021-01-22,Freeport,"Set via sx10 sta vdd7 and sm6 jan 22, 2021"
4,John Max,2873780,13411525,0.3,4.3,4,2023-02-20,GIWW,RESET TO MLLW/KTD value 0.3 ft 88/g18 (vrs obs)
...,...,...,...,...,...,...,...,...,...
23,Roseland Park,3256059.16,13832613.35,-0.1,3.9,4,2020-05-19,CEDAR BAYOU,NaN
24,Power Plant Outfall,3261170.781,13843708.805,-0.1,3.9,4,2020-05-19,CEDAR BAYOU,NaN
25,CEDAR BAYOU @ SH 146,3263949.882,13850155.681,-0.1,3.9,4,2020-05-19,CEDAR BAYOU,NaN
26,Dickinson Bayou,3249831.114,13737348.53,0.0,4,4,2020-04-03,Dickinson Bayou,NaN


In [39]:
# drop the first two rows since they don't contain anything of value.
df.drop([0, 1], inplace = True)
#Reset the index
df = df.reset_index(drop=True)

# Drop the index column that gets created
#
#df = df.drop("index", axis = 1, drop = True)

#Insert a column called GPT, Hypack needs this
df.insert(0, "GPT", "GPT")
#Reset the index
df = df.reset_index(drop=True)

#Drop the extra columns that are not needed
df = df.drop(df.iloc[:, 4:7], axis=1)
df = df.drop(df.iloc[:, 5:8], axis=1)

df

,GPT,Staff Name,Location,Unnamed: 2,Date Set or Checked
0,GPT,Brazos Harbor,3142320.1,13541514.3,2023-09-18
1,GPT,Freeport Gate (guillotine),3139199.6,13548074.5,2021-01-22
2,GPT,John Max,2873780,13411525,2023-02-20
3,GPT,Oyster Lake,2885722.11,13419985.54,2023-02-20
4,GPT,Mad Island Cut,2898569,13422758,2023-02-20
...,...,...,...,...,...
92,GPT,Roseland Park,3256059.16,13832613.35,2020-05-19
93,GPT,Power Plant Outfall,3261170.781,13843708.805,2020-05-19
94,GPT,CEDAR BAYOU @ SH 146,3263949.882,13850155.681,2020-05-19
95,GPT,Dickinson Bayou,3249831.114,13737348.53,2020-04-03


In [40]:
# rename the headers
Headers = ["GPT", "Name", "Easting", "Northing","Date"]
df = df.set_axis(Headers, axis = 1)
df

,GPT,Name,Easting,Northing,Date
0,GPT,Brazos Harbor,3142320.1,13541514.3,2023-09-18
1,GPT,Freeport Gate (guillotine),3139199.6,13548074.5,2021-01-22
2,GPT,John Max,2873780,13411525,2023-02-20
3,GPT,Oyster Lake,2885722.11,13419985.54,2023-02-20
4,GPT,Mad Island Cut,2898569,13422758,2023-02-20
...,...,...,...,...,...
92,GPT,Roseland Park,3256059.16,13832613.35,2020-05-19
93,GPT,Power Plant Outfall,3261170.781,13843708.805,2020-05-19
94,GPT,CEDAR BAYOU @ SH 146,3263949.882,13850155.681,2020-05-19
95,GPT,Dickinson Bayou,3249831.114,13737348.53,2020-04-03


In [41]:
# convert date format
df['Date'] = df['Date'].dt.strftime('%m/%d/%Y')

# Add Quotes around the Staff Names
df["Name"] = [f'"{j}"' for j in df["Name"]]

df

,GPT,Name,Easting,Northing,Date
0,GPT,"""Brazos Harbor """,3142320.1,13541514.3,09/18/2023
1,GPT,"""Freeport Gate (guillotine)""",3139199.6,13548074.5,01/22/2021
2,GPT,"""John Max""",2873780,13411525,02/20/2023
3,GPT,"""Oyster Lake""",2885722.11,13419985.54,02/20/2023
4,GPT,"""Mad Island Cut""",2898569,13422758,02/20/2023
...,...,...,...,...,...
92,GPT,"""Roseland Park""",3256059.16,13832613.35,05/19/2020
93,GPT,"""Power Plant Outfall""",3261170.781,13843708.805,05/19/2020
94,GPT,"""CEDAR BAYOU @ SH 146""",3263949.882,13850155.681,05/19/2020
95,GPT,"""Dickinson Bayou""",3249831.114,13737348.53,04/03/2020


In [42]:
#Export to target file
#TODO add Output_File variable and datestamp to filename ex: SWG_Tide_Staffs_20241124.tgt"
df.to_csv(WD + Output_File,sep='\t', header= False, index=False, quoting=csv.QUOTE_NONE)

#For now, configure the output filename here:
#df.to_csv(WD + 'SWG_Tide_Staffs_20241124.tgt',sep='\t', header= False, index=False, quoting=csv.QUOTE_NONE)

### Jupyter Notebooks have completely ruined me.